In [8]:
import paramiko

host = 'Midas'

config = paramiko.SSHConfig.from_path('/home/nick/.ssh/config')
host_config = config.lookup(host)
print(host_config)

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host_config['hostname'], key_filename=host_config['identityfile'], username=host_config['user'])

stdin, stdout, stderr = client.exec_command('ls -l', timeout=10)
ssh_output = stdout.read()
ssh_error = stderr.read()

print(str(ssh_output, ('utf8')), str(ssh_error, 'utf8'))


{'hostname': 'hpc.midasnetwork.us', 'user': 'nick_collier_midasnetwork_us', 'identityfile': ['/home/nick/.ssh/id_rsa_midas']}
total 12
drwxr-xr-x. 4 nick_collier_midasnetwork_us hepcep 4096 Oct 25  2019 repos
drwxr-xr-x. 4 nick_collier_midasnetwork_us hepcep 4096 Nov  5  2019 src
drwxr-xr-x. 2 nick_collier_midasnetwork_us hepcep 4096 Jan 21  2022 swift_test
 b''


In [9]:
# Execute command with Transport
import paramiko

# Using key agent to load keys and avoid entering key password
agent = paramiko.Agent()
keys = agent.get_keys()

host = 'Midas'
# found through trial and error
key = keys[3]

config = paramiko.SSHConfig.from_path('/home/nick/.ssh/config')
host_config = config.lookup(host)
print(host_config)

t = paramiko.Transport(host_config['hostname'])
# THIS FAILS FOR THE MIDAS KEY
# pkey = paramiko.RSAKey.from_private_key_file(host_config['identityfile'][0])
t.start_client()
t.auth_publickey(host_config['user'], key)
    
stdout_data = []
stderr_data = []
session = t.open_session()
session.exec_command('ls -l')
RECV_BYTES = 4096
while True:
    if session.exit_status_ready():
        if session.recv_ready():
            stdout_data.append(session.recv(RECV_BYTES))
        if session.recv_stderr_ready():
            stderr_data.append(session.recv_stderr(RECV_BYTES))
        break

print ('exit status: ', session.recv_exit_status())
print (str(b''.join(stdout_data), 'utf8'))
print (str(b''.join(stderr_data), 'utf8'))

session.close()
t.close() 

{'hostname': 'hpc.midasnetwork.us', 'user': 'nick_collier_midasnetwork_us', 'identityfile': ['/home/nick/.ssh/id_rsa_midas']}
exit status:  0
total 12
drwxr-xr-x. 4 nick_collier_midasnetwork_us hepcep 4096 Oct 25  2019 repos
drwxr-xr-x. 4 nick_collier_midasnetwork_us hepcep 4096 Nov  5  2019 src
drwxr-xr-x. 2 nick_collier_midasnetwork_us hepcep 4096 Jan 21  2022 swift_test




In [16]:
# Handler for interactive non-key based authentication
import getpass

def handler(title, instructions, prompt_list):
    # print(title, instructions)
    # print(f'Prompt: {prompt_list}')
    response = []
    if len(prompt_list) > 0:
        prompt, echo = prompt_list[0]
        if echo:
            response.append(input(str(prompt)))
        else:
            response.append(getpass.getpass(str(prompt)))

    return response


In [17]:
# Authenticate interactively password (e.g. on Theta)
import paramiko

host = 'Theta'

config = paramiko.SSHConfig.from_path('/home/nick/.ssh/config')
host_config = config.lookup(host)
print(host_config)

t = paramiko.Transport(host_config['hostname'])
t.start_client()
t.auth_interactive(host_config['user'], handler)
stdout_data = []
stderr_data = []
session = t.open_session()
session.exec_command('ls -l')
RECV_BYTES = 4096
while True:
    if session.exit_status_ready():
        if session.recv_ready():
            stdout_data.append(session.recv(RECV_BYTES))
        if session.recv_stderr_ready():
            stderr_data.append(session.recv_stderr(RECV_BYTES))
        break

print ('exit status: ', session.recv_exit_status())
print (str(b''.join(stdout_data), 'utf8'))
print (str(b''.join(stderr_data), 'utf8'))

session.close()
t.close() 


{'hostname': 'theta.alcf.anl.gov', 'user': 'ncollier', 'identityfile': ['/home/nick/.ssh/id_rsa']}
 


AuthenticationException: Authentication failed.

In [4]:
# Authenticate with custom handler - password + 2FA
import paramiko


# Midway doesn't use key - password + duo.
host = 'Midway2'

config = paramiko.SSHConfig.from_path('/home/nick/.ssh/config')
host_config = config.lookup(host)
print(host_config)

t = paramiko.Transport(host_config['hostname'])
t.start_client()
# Note can use auth_interactive_dumb without handler, but that
# seems to ignore whether or not response should be echo'd
t.auth_interactive(host_config['user'], handler)
stdout_data = []
stderr_data = []
session = t.open_session()
session.exec_command('ls -l')
RECV_BYTES = 4096
while True:
    if session.exit_status_ready():
        if session.recv_ready():
            stdout_data.append(session.recv(RECV_BYTES))
        if session.recv_stderr_ready():
            stderr_data.append(session.recv_stderr(RECV_BYTES))
        break

print ('exit status: ', session.recv_exit_status())
print (str(b''.join(stdout_data), 'utf8'))
print (str(b''.join(stderr_data), 'utf8'))

session.close()
t.close() 

{'hostname': 'midway2.rcc.uchicago.edu', 'user': 'ntcollie', 'identityfile': ['/home/nick/.ssh/id_rsa']}
Prompt: [('Password: ', False)]
Prompt: [('Duo two-factor login for ntcollie\n\nEnter a passcode or select one of the following options:\n\n 1. Duo Push to XXX-XXX-2212\n 2. Phone call to XXX-XXX-2212\n 3. SMS passcodes to XXX-XXX-2212\n\nPasscode or option (1-3): ', True)]
Prompt: []
exit status:  0
b'total 486\n-rw-rw-r--    1 ntcollie ntcollie      0 Dec  7 14:16 =0.1.0\n-rw-rw-r--    1 ntcollie ntcollie   9718 Dec  7 14:16 =1.0.5\ndrwxrwxr-x    4 ntcollie ntcollie   4096 Nov 10  2016 R\n-rw-rw-r--    1 ntcollie ntcollie  13666 Dec  2  2012 R-benchmark-25.R\n-rwxrw-r--    1 ntcollie ntcollie     74 Dec 19  2016 bars_modules.sh\ndrwxrwxr-x 7293 ntcollie ntcollie 262144 Feb 17  2020 data\n-rw-rw-r--    1 ntcollie ntcollie  55278 Nov 10  2016 downloads.html\ndrwxr-xr-x    3 ntcollie ntcollie   4096 Sep  4  2020 intel\n-rw-rw-r--    1 ntcollie ntcollie      0 Feb 17  2020 localParamF

In [60]:
# Test DB connection
import psycopg2

conn = psycopg2.connect("dbname=EQ_SQL user=collier port=11219 host=localhost")
cur = conn.cursor()
cur.execute("select * from eq_task_tags;")
cur.fetchall()



OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 11219?


In [1]:
# Port forwarding handler / server

# Copyright (C) 2003-2007  Robey Pointer <robeypointer@gmail.com>
#
# This file is part of paramiko.
#
# Paramiko is free software; you can redistribute it and/or modify it under the
# terms of the GNU Lesser General Public License as published by the Free
# Software Foundation; either version 2.1 of the License, or (at your option)
# any later version.
#
# Paramiko is distributed in the hope that it will be useful, but WITHOUT ANY
# WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
# A PARTICULAR PURPOSE.  See the GNU Lesser General Public License for more
# details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with Paramiko; if not, write to the Free Software Foundation, Inc.,
# 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301 USA.

"""
Sample script showing how to do local port forwarding over paramiko.

This script connects to the requested SSH server and sets up local port
forwarding (the openssh -L option) from a local port through a tunneled
connection to a destination reachable from the SSH server machine.
"""

import getpass
import os
import socket
import select
import socketserver

import sys
import paramiko

class ForwardServer(socketserver.ThreadingMixIn, socketserver.TCPServer):

    allow_reuse_address = True
    daemon_threads = True

class Handler(socketserver.BaseRequestHandler):
    def handle(self):
        try:
            chan = self.ssh_transport.open_channel(
                "direct-tcpip",
                (self.chain_host, self.chain_port),
                self.request.getpeername(),
            )
        except Exception as e:
            print(
                "Incoming request to %s:%d failed: %s"
                % (self.chain_host, self.chain_port, repr(e))
            )
            return
        if chan is None:
            print(
                "Incoming request to %s:%d was rejected by the SSH server."
                % (self.chain_host, self.chain_port)
            )
            return

        print(
            "Connected!  Tunnel open %r -> %r -> %r"
            % (
                self.request.getpeername(),
                chan.getpeername(),
                (self.chain_host, self.chain_port),
            )
        )
        while True:
            r, w, x = select.select([self.request, chan], [], [])
            if self.request in r:
                data = self.request.recv(1024)
                if len(data) == 0:
                    break
                chan.send(data)
            if chan in r:
                data = chan.recv(1024)
                if len(data) == 0:
                    break
                self.request.send(data)

        peername = self.request.getpeername()
        chan.close()
        self.request.close()
        print("Tunnel closed from %r" % (peername,))


In [7]:
# DB Access Using SSH Tunnel

# Connect to Bebop
import paramiko
import psycopg2
from threading import Thread

# Using key agent to load keys and avoid entering key password
agent = paramiko.Agent()
keys = agent.get_keys()

host = 'Bebop'
# found through trial and error
key = keys[0]

config = paramiko.SSHConfig.from_path('/home/nick/.ssh/config')
host_config = config.lookup(host)
print(host_config)

t = paramiko.Transport(host_config['hostname'])
t.start_client()
t.auth_publickey(host_config['user'], key)


# ssh -L :11219:beboplogin4.lcrc.anl.gov:57164 collier@beboplogin1.lcrc.anl.gov -N -C
# forward_tunnel(11219, 'beboplogin4.lcrc.anl.gov', 57164, transport=t)
class SubHander(Handler):
    chain_host = 'beboplogin4.lcrc.anl.gov'
    chain_port = 57164
    ssh_transport = t

fs = ForwardServer(("", 11219), SubHander, bind_and_activate=False)
fs.allow_reuse_address = True
fs.server_bind()
fs.server_activate()
th = Thread(target=fs.serve_forever)
th.start()
# forwarder = Thread(target=forward_tunnel, args=(11219, 'beboplogin4.lcrc.anl.gov', 57164, t))
# forwarder.start()

conn = psycopg2.connect("dbname=EQ_SQL user=collier port=11219 host=localhost")
cur = conn.cursor()
cur.execute("select * from eq_task_tags;")
print(f'select * from eq_task_tags: {cur.fetchall()}')

t.close()
fs.shutdown()


{'hostname': 'bebop.lcrc.anl.gov', 'user': 'collier', 'identityfile': ['/home/nick/.ssh/id_rsa']}
Connected!  Tunnel open ('127.0.0.1', 55606) -> ('140.221.70.7', 22) -> ('beboplogin4.lcrc.anl.gov', 57164)
select * from eq_task_tags: [(1, 'test')]
Tunnel closed from ('127.0.0.1', 55606)
